# Imports

In [1]:
SEED = 42
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
import random
import pandas as pd
import ast
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#######################################
np.random.seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-05-13 20:02:35.925018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747159355.936180   86722 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747159355.939120   86722 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747159355.948119   86722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747159355.948127   86722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747159355.948128   86722 computation_placer.cc:177] computation placer alr

# Loading the data

In [2]:
root = "data_final_project/KuaiRec 2.0/data/"
train_data = pd.read_parquet("./exports/train_data_feature_engineered.pq")
test_data = pd.read_parquet("./exports/test_data_feature_engineered.pq")

In [3]:
train_data.sample(5, random_state=SEED)

,user_id,video_duration,watch_ratio,feat_12,show_cnt,valid_play_cnt,follow_cnt,cancel_follow_cnt,user_cum_avg_watch_ratio,video_cum_avg_watch_ratio,cumulative_avg_watch_ratio,friend_watch_ratio
4496404,3292,7000,0.960143,0,1122549.0,603953.0,1125.0,0.0,0.818546,1.010236,0.828277,1.010236
9290251,6802,8600,1.260233,0,0.0,0.0,0.0,0.0,1.140637,0.922110,1.260233,0.728488
6632732,4807,9140,1.171116,0,275807.0,119066.0,917.0,0.0,0.837763,0.900782,0.991791,0.900782
9236339,6770,5943,0.474340,0,7033.0,4815.0,0.0,0.0,0.803386,1.271882,0.871679,1.271882
9379302,6867,8213,0.265189,0,0.0,0.0,0.0,0.0,0.744441,1.033243,0.836436,1.033243


In [4]:
def to_dataset(df: pd.DataFrame) -> pd.DataFrame:
    target = "watch_ratio"
    exclude = ["user_id", target]
    features = [col for col in df.columns if col not in exclude]
    X, y = df[features].values, df[target].values
    return X, y

X_train, y_train = to_dataset(train_data)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=SEED)
X_test, y_test = to_dataset(test_data)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

I0000 00:00:1747159360.264457   86722 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13466 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mean_squared_error',
    metrics=['mae']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=1024,
    callbacks=[keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)]
)

Epoch 1/10


2025-05-13 20:02:41.422409: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


8597/8601 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4493 - mae: 0.4601

2025-05-13 20:02:59.842458: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


8601/8601 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.4493 - mae: 0.4601 - val_loss: 0.3621 - val_mae: 0.3960
Epoch 2/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3653 - mae: 0.4062 - val_loss: 0.3604 - val_mae: 0.3952
Epoch 3/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.3626 - mae: 0.4033 - val_loss: 0.3608 - val_mae: 0.3959
Epoch 4/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3617 - mae: 0.4022 - val_loss: 0.3606 - val_mae: 0.3961
Epoch 5/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3611 - mae: 0.4015 - val_loss: 0.3602 - val_mae: 0.3965
Epoch 6/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3608 - mae: 0.4011 - val_loss: 0.3601 - val_mae: 0.3966
Epoch 7/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3605 - mae: 0.4008 - val_loss: 0.3600 - val_mae: 0.3964
Epoch 8/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3603 - mae: 0.4006 - val_loss: 0.3602 - val_mae: 0.3966
Epoch 9/10
8601/8601 ━━━━━━━━━━━━━━━━━━━━ 1

In [7]:
baseline_predictions = np.full_like(y_test, np.mean(y_test), dtype=np.float32)
baseline_test_mae = np.mean(np.abs(baseline_predictions - y_test))
print(f"Baseline MAE: {baseline_test_mae:.4f}")

Baseline MAE: 0.4230


### EVALUATE THE MODEL

In [8]:
y_pred = model.predict(X_test, batch_size=2048).flatten()
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.4f}")

 334/1890 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step

2025-05-13 20:05:43.904770: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


1890/1890 ━━━━━━━━━━━━━━━━━━━━ 1s 479us/step
Model MAE: 0.3013


In [10]:
sample_df = pd.DataFrame({"actual": y_test, "predicted": y_pred}).sample(10, random_state=SEED)
print(sample_df)

           actual  predicted
1368882  0.099968   0.719875
2669510  0.142961   0.164430
2995806  1.526053   1.124395
556239   0.618283   1.069494
105496   0.997921   1.074542
1020242  0.967097   0.868689
2744890  1.187240   1.175679
142460   0.883650   0.942452
590581   1.384375   1.160124
3532687  1.122799   0.801645
